### Import Lib and Data Preprocess

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical
import librosa
import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.layers import Bidirectional, TimeDistributed, Conv2D, MaxPooling2D, Input, GRU, Dense, Activation, Dropout, Reshape, Permute, MaxPooling1D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


### Load the extracted features

In [ ]:
X=np.load('/content/drive/MyDrive/ML_S/Tri train/X.npy')
y=np.load('/content/drive/MyDrive/ML_S/Tri train/y.npy')

### Split Data

In [ ]:
## Splitting the data into 60% unlabeled 10% testing and 30% training
X_train, X_un, y_train, y_un = train_test_split(X, y, test_size=0.6,random_state=1)
X_label, X_test, y_label, y_test = train_test_split(X_train, y_train, test_size=0.25,random_state=1)

### Generate Models

In [ ]:
clf=[[],[],[]]
callbacks=[[],[],[]]

### MODEL1: CRNN ###

seq = keras.Sequential(
    [
        layers.Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape=(16,8,1) ),
        layers.MaxPool2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), padding = "same", activation = "tanh"),
        layers.MaxPool2D(pool_size=(2, 2)),
        layers.Reshape((8, 128)),
        layers.LSTM(32),
        layers.Flatten(),
        layers.Dense(32),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax'),
    ]
)
opt = keras.optimizers.Adam(learning_rate=0.01)
seq.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
callbacks[0] = [ModelCheckpoint('/content/drive/MyDrive/ML_S/Tri train/CNN+LSTM.h5', monitor='val_loss', mode='min', save_best_only=True),EarlyStopping(monitor = 'val_loss',min_delta = 0,patience = 40,restore_best_weights = True)]
clf[0].append(seq)

### MODEL2: LSTM ###
input_dim = (16, 8)
dropout=0.1
n_classes=10
learning_rate = 0.001

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=input_dim,dropout=dropout))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))
opt = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
callbacks[1] = [ModelCheckpoint('/content/drive/MyDrive/ML_S/Tri train/base_LSTM.h5', monitor='val_loss', mode='min', save_best_only=True),EarlyStopping(monitor = 'val_loss',min_delta = 0,patience = 40,restore_best_weights = True)]

clf[1].append(model)

### Model3 MLP ###
input_dim = (128,)
dropout=0.1
n_classes=10
learning_rate = 0.0001
num_classes = 10

model = Sequential()

model.add(Dense(256, input_shape=input_dim ))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
callbacks[2] = [ModelCheckpoint('/content/drive/MyDrive/ML_S/Tri train/MLP.h5', monitor='val_loss', mode='min', save_best_only=True)]

clf[2].append(model)

### Train models

Bootstrap the sample data and train the model

In [ ]:
s=pd.DataFrame([])
iter=[]
val_acc=[[],[],[]]
train_acc=[[],[],[]]
X_Label=[[],[],[]]
error=[[],[],[]]
X_Test=[[],[],[]]
X_Un=[[],[],[]]
y_Label=[[],[],[]]
y_Un=[[],[],[]]
iter.append(1)

In [ ]:
for i in range(3):
  if(i==0):
    print("CRNN Model:\n")
    X_Label[i] = X_label.reshape(len(X_label), 16, 8,1)
    y_Label[i]=y_label
    X_Test[i] = X_test.reshape(len(X_test), 16, 8,1)
    X_Un[i]= X_un.reshape(len(X_un), 16, 8,1)
    y_Un[i]=y_un
    X_resample, y_resample = sklearn.utils.resample(X_Label[i],y_Label[i])  # BootstrapSample(L)
    X__train, X__val, y__train, y__val = train_test_split(X_resample, y_resample, test_size=0.25)
    clf[i].fit(X__train,y__train,validation_data = (X__val, y__val),epochs = 300, batch_size = 500,shuffle=True,callbacks=callbacks[i])  # Learn(Si)
    
  if(i==1):
    print("\n LSTM Model:\n")
    X_Label[i] = X_label.reshape(len(X_label), 16, 8)
    y_Label[i]=y_label
    X_Test[i] = X_test.reshape(len(X_test), 16, 8)
    X_Un[i] = X_un.reshape(len(X_un), 16, 8)
    y_Un[i]=y_un
    X_resample, y_resample = sklearn.utils.resample(X_Label[i],y_Label[i])  # BootstrapSample(L)
    X__train, X__val, y__train, y__val = train_test_split(X_resample, y_resample, test_size=0.25)
    clf[i].fit(X__train,y__train,validation_data = (X__val, y__val),epochs = 300, batch_size = 500,shuffle=True,callbacks=callbacks[i])  # Learn(Si)

  if(i==2):
    print("\n Multi Layer Perceptron Model:\n")
    X_Label[i]=X_label.reshape(len(X_label),128)
    y_Label[i]=y_label
    X_Test[i] = X_test.reshape(len(X_test),128)
    X_Un[i] = X_un.reshape(len(X_un),128)
    y_Un[i]=y_un
    X_resample, y_resample = sklearn.utils.resample(X_Label[i],y_Label[i])  # BootstrapSample(L)
    X__train, X__val, y__train, y__val = train_test_split(X_resample, y_resample, test_size=0.25)
    clf[i].fit(X__train,y__train,validation_data = (X__val, y__val),epochs = 300, batch_size = 32,shuffle=True,callbacks=callbacks[i])  # Learn(Si)
    

  accuracy1=clf[i].evaluate(X__val,y__val)
  print("val_accuracy=",accuracy1)
  val_acc[i].append(accuracy1[1]*100)
  accuracy1=clf[i].evaluate(X__train,y__train)
  train_acc[i].append(accuracy1[1]*100)
  print("train accuracy=",accuracy1)
  error[i].append(0.5)


CRNN Model:

Epoch 1/300
4/4 [==============================] - 3s 377ms/step - loss: 2.2867 - accuracy: 0.1093 - val_loss: 2.1172 - val_accuracy: 0.1695
Epoch 2/300
4/4 [==============================] - 1s 213ms/step - loss: 2.1025 - accuracy: 0.1866 - val_loss: 1.9851 - val_accuracy: 0.2748
Epoch 3/300
4/4 [==============================] - 1s 220ms/step - loss: 1.9920 - accuracy: 0.2574 - val_loss: 1.9081 - val_accuracy: 0.3267
Epoch 4/300
4/4 [==============================] - 1s 216ms/step - loss: 1.9388 - accuracy: 0.2851 - val_loss: 1.7986 - val_accuracy: 0.3664
Epoch 5/300
4/4 [==============================] - 1s 211ms/step - loss: 1.8011 - accuracy: 0.3223 - val_loss: 1.7922 - val_accuracy: 0.3374
Epoch 6/300
4/4 [==============================] - 1s 206ms/step - loss: 1.7445 - accuracy: 0.3493 - val_loss: 1.6668 - val_accuracy: 0.4153
Epoch 7/300
4/4 [==============================] - 1s 206ms/step - loss: 1.6379 - accuracy: 0.4038 - val_loss: 1.5960 - val_accuracy: 0.4229


#### Test accuracy of base models by majority voting

In [ ]:
iteration=1
y_pred_test=[[]]*3
  
for i in range(3):
  y_pred_test[i]=clf[i].predict(X_Test[i])

avg_pred_test=np.add(y_pred_test[0],y_pred_test[1],y_pred_test[2])/3
y_pred_test= np.argmax(avg_pred_test, axis=1)
y_t=np.argmax(y_test,axis=1)
correct=0
for p in range(len(y_test)):
  if (y_pred_test[p] == y_t[p]):
    correct=correct+1
acc=correct*100/len(y_t)
test_acc.append(acc)
print("Test accuracy after iteration",iteration,"=",acc)

Test accuracy after iteration 1 = 70.21764032073311


## Tri-training with disagreement

In [ ]:
update = [1,1,1]
error_prev=[0.5,0.5,0.5]
error_current=[0.5,0.5,0.5]
label_added=[[0],[0],[0]]
labeled=[[],[],[]]
unlabeled=[[],[],[]]
percent_correct=[[0],[0],[0]]
iteration=1
test_acc=[]

In [ ]:
while (iteration<5 and sum(update)):
  X_pseudo_label = [[], [], []]
  y_pseudo_label = [[], [], []]
  
  ## Step 1: Create pseudo labeled data
  print("Pseudo Label Step")
  error_prev=error_current
  for i in range(3):
    print("i=",i,"Iteration=",iteration,"\nlabeled=",len(X_Label[i]),"unlabeled=",len(X_Un[i]))
    labeled[i].append(len(X_Label[i]))
    unlabeled[i].append(len(X_Un[i]))
    j, k = np.delete(np.array([0, 1, 2]), i)

    ## measure error of jth and kth model: if percentage of same wrong predictions made by both models are higher than threshold then tri training is stopped
    j_pred=clf[j].predict(X_Label[j])
    l_j = np.argmax(j_pred, axis=1)
    l_j=np.array(l_j)
    k_pred=clf[k].predict(X_Label[k])
    l_k = np.argmax(k_pred, axis=1)
    l_k=np.array(l_k)
    y_=np.argmax(y_Label[i],axis=1)
    wrong_index =np.logical_and(l_k != y_, l_j==l_k)
    error_current[i]=sum(wrong_index)/sum(l_j==l_k)
    print("error=",error_current[i])
    error[i].append(error_current[i])

    if (error_current[i]<=error_prev[i] and sum(update)):
      update[i]=1
      y_pred_j=clf[j].predict(X_Un[j])
      y_pred_k=clf[k].predict(X_Un[k])
      y_pred_i=clf[i].predict(X_Un[i])
      
      l_correct = []
      ## calculate number of unlabeled samples satisfying the tri training condition to be considered as a pseudo label
      y_u=np.argmax(y_Un[i],axis=1)
      correct_label=0
      for p in range(len(X_Un[i])):
        for c in range(10):
          if (y_pred_j[p][c]>=0.6 and y_pred_k[p][c]>=0.6 and y_pred_i[p][c]<0.3):
            X_pseudo_label[i].append(X_Un[i][p])
            y_pseudo_label[i].append(c)
            l_correct.append(p)
            if (y_u[p]==c): correct_label=correct_label+1
      percent_correct[i].append(correct_label*100/len(X_pseudo_label[i]))
      print("\nLabels added",len(X_pseudo_label[i])," % correct=",correct_label*100/len(X_pseudo_label[i]))
      label_added[i].append(len(X_pseudo_label[i]))
    
    else: 
      update[i]=0
      label_added[i].append(0)
      percent_correct.append(0)

  ## Step 2: Update the models: Use pseudo labeled data with labeled for re training the models
  print("Updation step")
  if (sum(update)):
    X_pseudo_label=np.array(X_pseudo_label)
    y_pseudo_label=np.array(y_pseudo_label)
    for i in range(3):
      if (len(X_pseudo_label[i])!=0):
        update[i]=1
        X_Label[i]=np.array(X_Label[i])
        y_Label[i]=np.array(y_Label[i])
        y_l=np.argmax(y_Label[i],axis=1)

        ## create temperary X and y to re train the model
        X_temp=np.append(X_Label[i],X_pseudo_label[i],axis=0)
        y_temp=np.append(y_l,y_pseudo_label[i],axis=0)
        y_temp=to_categorical(y_temp,num_classes=10)

        # re train the model

        X__train, X__val, y__train, y__val = train_test_split(X_temp, y_temp, test_size=0.25)
        if (i is not 2):
          clf[i].fit(X__train,y__train,validation_data = (X__val, y__val),epochs = 300, batch_size = 500,shuffle=True,callbacks=callbacks[i])
        else: clf[i].fit(X__train,y__train,validation_data = (X__val, y__val),epochs = 300, batch_size = 32,shuffle=True,callbacks=callbacks[i])

        accuracy1=clf[i].evaluate(X__val,y__val)
        print("val_accuracy=",accuracy1)
        val_acc[i].append(accuracy1[1]*100)
        accuracy1=clf[i].evaluate(X__train,y__train)
        train_acc[i].append(accuracy1[1]*100)
        print("train accuracy=",accuracy1)
      else:
        update[i]=0
        val_acc[i].append(0)
        train_acc[i].append(0)
  
  ## Step 3: Test the data via majority voting
  print("Testing Step")
  y_pred_test=[[]]*3
  
  for i in range(3):
    y_pred_test[i]=clf[i].predict(X_Test[i])

  avg_pred_test=np.add(y_pred_test[0],y_pred_test[1],y_pred_test[2])/3
  y_pred_test= np.argmax(avg_pred_test, axis=1)
  y_t=np.argmax(y_test,axis=1)
  correct=0
  for p in range(len(y_test)):
    if (y_pred_test[p] == y_t[p]):
      correct=correct+1
  acc=correct*100/len(y_t)
  test_acc.append(acc)
  print("Test accuracy after iteration",iteration,"=",acc)

  iter.append(iteration)
  iteration=iteration+1

Pseudo Label Step
i= 0 Iteration= 1 
labeled= 2619 unlabeled= 5240
error= 0.08098223615464994

Labels added 376  % correct= 71.01063829787235
i= 1 Iteration= 1 
labeled= 2619 unlabeled= 5240
error= 0.10540254237288135

Labels added 203  % correct= 72.9064039408867
i= 2 Iteration= 1 
labeled= 2619 unlabeled= 5240
error= 0.07953340402969247

Labels added 308  % correct= 78.8961038961039
Updation step
Epoch 1/300


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


5/5 [==============================] - 1s 198ms/step - loss: 1.2304 - accuracy: 0.6607 - val_loss: 1.2383 - val_accuracy: 0.6128
Epoch 2/300
5/5 [==============================] - 1s 184ms/step - loss: 1.0524 - accuracy: 0.6607 - val_loss: 1.0938 - val_accuracy: 0.6662
Epoch 3/300
5/5 [==============================] - 1s 184ms/step - loss: 0.9469 - accuracy: 0.6857 - val_loss: 1.0794 - val_accuracy: 0.6569
Epoch 4/300
5/5 [==============================] - 1s 185ms/step - loss: 0.8684 - accuracy: 0.7115 - val_loss: 1.0269 - val_accuracy: 0.6529
Epoch 5/300
5/5 [==============================] - 1s 191ms/step - loss: 0.7662 - accuracy: 0.7489 - val_loss: 1.0572 - val_accuracy: 0.6689
Epoch 6/300
5/5 [==============================] - 1s 185ms/step - loss: 0.6809 - accuracy: 0.7747 - val_loss: 0.9974 - val_accuracy: 0.6822
Epoch 7/300
5/5 [==============================] - 1s 183ms/step - loss: 0.5972 - accuracy: 0.7983 - val_loss: 0.9424 - val_accuracy: 0.7143
Epoch 8/300
5/5 [========

In [ ]:
s['CRNN train acc']=train_acc[0]
s['CRNN val acc']=val_acc[0]
s['CRNN_pseudo_label']=label_added[0]
s['CRNN % correct added']=percent_correct[0]
s['LSTM train acc']=train_acc[1]
s['LSTM val acc']=val_acc[1]
s['LSTM_pseudo_label']=label_added[1]
s['LSTM % correct added']=percent_correct[1]
s['MLP train acc']=train_acc[2]
s['MLP val acc']=val_acc[2]
s['MLP_pseudo_label']=label_added[2]
s['MLP % correct added']=percent_correct[2]
s['test acc']=test_acc

In [ ]:
s

,CRNN train acc,CRNN val acc,CRNN_pseudo_label,CRNN % correct added,LSTM train acc,LSTM val acc,LSTM_pseudo_label,LSTM % correct added,MLP train acc,MLP val acc,MLP_pseudo_label,MLP % correct added,test acc
0,89.714867,73.893130,0,0.000000,93.890023,85.190839,0,0.000000,87.423623,76.946563,0,0.000000,0.000000
1,84.772933,71.428573,376,71.010638,89.083177,75.212467,203,72.906404,95.216399,76.502734,308,78.896104,74.226804
2,78.514713,73.389357,236,66.525424,86.946279,82.485878,211,69.668246,97.043139,85.755813,132,67.424242,74.455899
3,85.418540,78.272605,344,77.034884,84.735811,82.697946,107,67.289720,98.576337,87.187040,97,68.041237,75.372279
4,80.066288,79.545456,197,72.081218,86.074144,82.051283,187,71.122995,98.769081,90.251106,89,59.550562,73.768614


In [ ]:
s.to_csv("/content/drive/MyDrive/ML_S/Tri train/tri.csv")